# ED example

The exercise was adapted from [[1]](#R1). Exercise 14.1.1.

## a)

We want to minimize the cost of supplying a 850 MW load using 3 generators with these characteristics:

1. $P_1^{max}=600$ MW, $P_1^{min}=150$ MW, $H_1=510+7.2\cdot P_1+0.00142 \cdot P_1^2$
2. $P_2^{max}=400$ MW, $P_2^{min}=100$ MW, $H_2=310+7.85\cdot P_2+0.00194 \cdot P_2^2$
3. $P_3^{max}=200$ MW, $P_3^{min}=50$ MW, $H_3=78+7.97\cdot P_3+0.00482 \cdot P_3^2$

With the fuel costs being $FC_1=1.1$, $FC_2=1$ and $FC_3=1$, the cost functions become:


- $C_1(P_1)=561+7.92\cdot P_1+0.001562 \cdot P_1^2$ \$/h
- $C_2(P_2)=310+7.85\cdot P_2+0.00194 \cdot P_2^2$ \$/h
- $C_3(P_3)=78+7.97\cdot P_3+0.00482 \cdot P_3^2$ \$/h



In [1]:
# import the Pyomo solver
from pyomo.environ import *

In [2]:
# define a new model
model = ConcreteModel()

# declare decision variables for three units
model.P1 = Var(domain=NonNegativeReals)
model.P2 = Var(domain=NonNegativeReals)
model.P3 = Var(domain=NonNegativeReals)

In [3]:
# declare objective
model.objective = Objective(
    expr = (561+7.92*model.P1+0.001562*model.P1**2+
            310+7.85*model.P2+0.00194*model.P2**2+
            78+7.97*model.P3+0.00482*model.P3**2),
    sense = minimize)

In [4]:
# declare constraints for demand
model.demand = Constraint(expr = model.P1+model.P2+model.P3==850)

In [5]:
# enable to save the dual variables
model.dual = Suffix(direction=Suffix.IMPORT)

In [6]:
# solve the model
SolverFactory('ipopt').solve(model, tee=False)

In [7]:
print("Cost = ", model.objective())
print("P1 = ", model.P1(), " MW")
print("P2 = ", model.P2(), " MW")
print("P3 = ", model.P3(), " MW")
print("Incremental cost = ", model.dual.get(model.demand), " $")

Cost =  8194.356121270199
P1 =  393.16983694477966  MW
P2 =  334.60375531355845  MW
P3 =  122.22640774166184  MW
Incremental cost =  9.148262570609143  $


## b)

With the $FC_1$ reduced to 0.9, the cost function of unit 1 changes to:

$$C_1(P_1)=459+6.48\cdot P_1+0.00128 \cdot P_1^2\ \$/h$$ 

We first solve without any constraints:

In [8]:
# define a new model
model = ConcreteModel()

# declare decision variables for three units
model.P1 = Var(domain=NonNegativeReals)
model.P2 = Var(domain=NonNegativeReals)
model.P3 = Var(domain=NonNegativeReals)

# declare objective
model.objective = Objective(
    expr = (459+6.48*model.P1+0.00128*model.P1**2+
            310+7.85*model.P2+0.00194*model.P2**2+
            78+7.97*model.P3+0.00482*model.P3**2),
    sense = minimize)

# declare constraints for demand
model.demand = Constraint(expr = model.P1+model.P2+model.P3==850)

In [9]:
# enable to save the dual variables
model.dual = Suffix(direction=Suffix.IMPORT)

# solve the model
SolverFactory('ipopt').solve(model, tee=False)

print("Cost = ", model.objective())
print("P1 = ", model.P1(), " MW")
print("P2 = ", model.P2(), " MW")
print("P3 = ", model.P3(), " MW")
print("Incremental cost = ", model.dual.get(model.demand), " $")

Cost =  7223.385813948322
P1 =  705.1467484216384  MW
P2 =  112.15867937582065  MW
P3 =  32.69457220254095  MW
Incremental cost =  8.285175675955866  $


We see that units 1+3 are not within their limits. We re-solve adding the generator limits.

In [10]:
# define a new model
model = ConcreteModel()

# declare decision variables for three units
model.P1 = Var(domain=NonNegativeReals)
model.P2 = Var(domain=NonNegativeReals)
model.P3 = Var(domain=NonNegativeReals)

# declare objective
model.objective = Objective(
    expr = (459+6.48*model.P1+0.00128*model.P1**2+
            310+7.85*model.P2+0.00194*model.P2**2+
            78+7.97*model.P3+0.00482*model.P3**2),
    sense = minimize)

# declare constraints for demand and generation
model.demand = Constraint(expr = model.P1+model.P2+model.P3==850)
model.P1max = Constraint(expr = model.P1 <= 600)
model.P2max = Constraint(expr = model.P2 <= 400)
model.P3max = Constraint(expr = model.P3 <= 200)
model.P1min = Constraint(expr = model.P1 >= 150)
model.P2min = Constraint(expr = model.P2 >= 100)
model.P3min = Constraint(expr = model.P3 >= 50)

In [11]:
# enable to save the dual variables
model.dual = Suffix(direction=Suffix.IMPORT)

# solve the model
SolverFactory('ipopt').solve(model, tee=False)

print("Cost = ", model.objective())
print("P1 = ", model.P1(), " MW")
print("P2 = ", model.P2(), " MW")
print("P3 = ", model.P3(), " MW")
print("Incremental cost = ", model.dual.get(model.demand), " $")

Cost =  7252.8303220859025
P1 =  600.0000059955258  MW
P2 =  187.1301732261155  MW
P3 =  62.8698207783586  MW
Incremental cost =  8.576065072086976  $


We now see all the units are within limits. However, we have a higher cost compared to the uncostrained problem.

## c)

Finally, we add the losses function and all the constraints:

$$P_{\mathrm{loss}}=0.00003 P_{G1}^{2}+0.00009 P_{G2}^{2}+0.00012 P_{G3}^{2}$$

In [12]:
# define a new model
model = ConcreteModel()

# declare decision variables for three units
model.P1 = Var(domain=NonNegativeReals)
model.P2 = Var(domain=NonNegativeReals)
model.P3 = Var(domain=NonNegativeReals)

# declare objective
model.objective = Objective(
    expr = (561+7.92*model.P1+0.001562*model.P1**2+
            310+7.85*model.P2+0.00194*model.P2**2+
            78+7.97*model.P3+0.00482*model.P3**2),
    sense = minimize)

# declare constraints for demand and generation
model.demand = Constraint(expr = model.P1+model.P2+model.P3==850+
                          0.00003*model.P1**2+0.00009*model.P2**2+0.00012*model.P3**2)
model.P1max = Constraint(expr = model.P1 <= 600)
model.P2max = Constraint(expr = model.P2 <= 400)
model.P3max = Constraint(expr = model.P3 <= 200)
model.P1min = Constraint(expr = model.P1 >= 150)
model.P2min = Constraint(expr = model.P2 >= 100)
model.P3min = Constraint(expr = model.P3 >= 50)

# enable to save the dual variables
model.dual = Suffix(direction=Suffix.IMPORT)

# solve the model
SolverFactory('ipopt').solve(model, tee=False)

print("Cost = ", model.objective())
print("P1 = ", model.P1(), " MW")
print("P2 = ", model.P2(), " MW")
print("P3 = ", model.P3(), " MW")
print("Incremental cost = ", model.dual.get(model.demand), " $")

Cost =  8344.592723075131
P1 =  435.19842086273917  MW
P2 =  299.9699666182852  MW
P3 =  130.66058332744208  MW
Incremental cost =  9.528363594150708  $


## References
<a id="R1">[1]</a> Wood, A. J., Wollenberg, B. F., & Sheble, G. B. (2014). Power generation, operation, and control (3rd ed.). Wiley-IEEE Press.